In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings(action='ignore')

In [1]:
import os
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import joblib 
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from datetime import datetime as dt

# 회귀분석
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from math import sqrt

import warnings
warnings.filterwarnings("ignore")

## 데이터 불러오기

In [14]:
data_dir = 'C:/Users/KimMinyoung/Desktop/깃헙관련/빅콘git/data/total/'

In [15]:
data_list = os.listdir(data_dir)
#print(len(data_list))
#print(data_list)

In [16]:
#년도별 저장
data_list_2016 = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-1].startswith('2016') ]
data_2016 = [pd.read_csv(os.path.join(data_dir,data_list_2016[x]),encoding='cp949') for x in range(len(data_list_2016))]
data_list_2017 = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-1].startswith('2017') ]
data_2017 = [pd.read_csv(os.path.join(data_dir,data_list_2017[x]),encoding='cp949') for x in range(len(data_list_2017))]
data_list_2018 = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-1].startswith('2018') ]
data_2018 = [pd.read_csv(os.path.join(data_dir,data_list_2018[x]),encoding='cp949') for x in range(len(data_list_2018))]
data_list_2019 = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-1].startswith('2019') ]
data_2019 = [pd.read_csv(os.path.join(data_dir,data_list_2019[x]),encoding='cp949') for x in range(len(data_list_2019))]
data_list_2020 = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-1].startswith('2020') ]
data_2020 = [pd.read_csv(os.path.join(data_dir,data_list_2020[x]),encoding='cp949') for x in range(len(data_list_2020))]
#print('2020년 데이터: \n', data_2020)

#항목별 저장
data_list_single_hitter = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('개인타자')]
data_single_hitter = [pd.read_csv(os.path.join(data_dir, data_list_single_hitter[x]), encoding='cp949') for x in range(len(data_list_single_hitter))]
data_list_single_pitcher = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('개인투수')]
data_single_pitcher = [pd.read_csv(os.path.join(data_dir, data_list_single_pitcher[x]), encoding='cp949') for x in range(len(data_list_single_pitcher))]
data_list_games = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('경기')]
data_games = [pd.read_csv(os.path.join(data_dir, data_list_games[x]), encoding='cp949') for x in range(len(data_list_games))]
data_list_player_enroll = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('등록선수')]
data_player_enroll = [pd.read_csv(os.path.join(data_dir, data_list_player_enroll[x]), encoding='cp949') for x in range(len(data_list_player_enroll))]
data_list_players = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('선수')]
data_players = [pd.read_csv(os.path.join(data_dir, data_list_players[x]), encoding='cp949') for x in range(len(data_list_players))]
data_list_teams = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('팀')]
data_teams = [pd.read_csv(os.path.join(data_dir, data_list_teams[x]), encoding='cp949') for x in range(len(data_list_teams))]
data_list_team_hitter = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('팀타자')]
data_team_hitter = [pd.read_csv(os.path.join(data_dir, data_list_team_hitter[x]), encoding='cp949') for x in range(len(data_list_team_hitter))]
data_list_team_pitcher = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('팀투수')]
data_team_pitcher = [pd.read_csv(os.path.join(data_dir, data_list_team_pitcher[x]), encoding='cp949') for x in range(len(data_list_team_pitcher))]
#print('팀투수 데이터:\n', data_team_pitcher)

### EDA

* eda
* lgbm class
* lgbm experiment class

### 투수

In [17]:
from sklearn.preprocessing import LabelEncoder

def team_pitcher_eda(year, team_name):
    year_index = year - 2016 
    
    # team_pitcher data에 해당 경기의 해당 팀의 홈 어웨이 여부 column 추가
    data_games_year = data_games[year_index]
    data_games_year = data_games_year[['G_ID', 'VISIT_KEY', 'HOME_KEY']]
    data_games_year = data_games_year.set_index('G_ID')
    
    data_team_pitcher_year = data_team_pitcher[year_index]
    data_team_pitcher_year = pd.merge(data_team_pitcher_year, data_games_year, how='left', on=['G_ID'])

    data_team_pitcher_year_team = data_team_pitcher_year[data_team_pitcher_year.T_ID == team_name]
    df = data_team_pitcher_year_team
    df['HOME_KEY'] = df['HOME_KEY'].map(lambda x: 1 if x == team_name else 0 )
    df = df.drop(columns = ['VISIT_KEY', 'GDAY_DS'])
    
    # LabelEncoding
    encoder = LabelEncoder()

    df['VS_T_ID'] = encoder.fit_transform(df['VS_T_ID']) #상대편
    df['TB_SC'] = encoder.fit_transform(df['TB_SC']) #이닝 초/말
    df['WLS'] = encoder.fit_transform(df['WLS']) #승패무 여부
    
    
    ### 이부분은 모든 EDA에서 동일하게 작성
    
    df['H1']=df['HIT']-df['H2']-df['H3']-df['HR']

    #df['G_ID'] = pd.Series(dt)
    df['year']=df['G_ID'].str.slice(0,4).astype(int)
    df['month']=df['G_ID'].str.slice(4,6).astype(int)
    
    df["FIP"] = ((13*df.HR+3*(df.BB+df.HP-df.IB)-2*df.KK)/(df.INN2/3)) +3.2
    df["BABIP"] = (df.HIT-df.HR)/(df.AB-df.KK-df.HR+df.SF)
    df['SLG']=(df['HIT']+df['HP']+df['BB'])/(df['AB']+df['BB']+df['HP']+df['SF'])
    df['OBA']=(df['H1']+2*df['H2']+3*df['H3']+4*df['HR'])/df['AB']
    df['W_OPS']=0.57* df['SLG']+0.43*df['OBA']

    # 시계열 데이터(경기당 방어율 계산)이기 때문에 경기 코드를 index로
    df = df.set_index('G_ID')

    # 경기당 방어율 column 생성
    df['ERA'] = df['ER'] * 27 / df['INN2']

    df = df.drop(columns=['T_ID'])
    df_year_team_name = df
    
    return df_year_team_name
        

In [18]:
df = team_pitcher_eda(2016, 'SS')
df

,VS_T_ID,HEADER_NO,TB_SC,CG_CK,WLS,HOLD,INN2,BF,PA,AB,...,HOME_KEY,H1,year,month,FIP,BABIP,SLG,OBA,W_OPS,ERA
G_ID,,,,,,,,,,,,,,,,,,,,,
20160401OBSS0,6,0,0,0,1,0,27,147,37,31,...,1,6,2016,4,6.755556,0.222222,0.333333,0.451613,0.384194,4.00
20160402OBSS0,6,0,0,0,2,0,27,160,42,38,...,1,5,2016,4,4.644444,0.354839,0.380952,0.552632,0.454774,6.00
20160405SSKT0,2,0,1,0,1,0,24,159,42,32,...,0,6,2016,4,6.325000,0.370370,0.452381,0.562500,0.499732,9.00
20160406SSKT0,2,0,1,0,2,0,27,138,38,36,...,0,2,2016,4,6.866667,0.148148,0.236842,0.500000,0.350000,6.00
20160407SSKT0,2,0,1,0,2,1,27,139,34,30,...,0,3,2016,4,2.533333,0.190476,0.235294,0.166667,0.205784,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20161003LGSS0,3,0,0,0,1,0,27,188,46,40,...,1,9,2016,10,6.088889,0.393939,0.422222,0.625000,0.509417,8.00
20161004LGSS0,3,0,0,0,2,1,27,148,35,32,...,1,7,2016,10,4.977778,0.318182,0.323529,0.468750,0.385974,4.00
20161005HTSS0,1,0,0,0,1,0,27,140,38,32,...,1,8,2016,10,2.866667,0.423077,0.400000,0.437500,0.416125,3.00


In [19]:
def make_y_next(df, rm_range, target):
    df['y_next'] = 0
    
    t_dict = {}
    t_list = []
    count = 0
    f_count = 0
    for i, v in enumerate(df[target]):

        if df[target].index[i] == df[target].index[-1]:
            t_dict[str(f_count)] = round(np.mean(np.array(t_list)),5)

        t_list.append(v)
        count += 1

        if count == rm_range:
            t_dict[str(f_count)] = round(np.mean(np.array(t_list)),5)
            count = 0
            t_list = []
            f_count += 1

    rm_dict = {}
    rm_list = []
    rm_count = 0
    rm_fcount = 0

    for i, v in enumerate(df['y_next']):
        if df['y_next'].index[i] == df['y_next'].index[-1]:
            rm_dict[str(rm_fcount)] = rm_list

        rm_list.append(i)
        rm_count += 1

        if rm_count == rm_range:
            rm_dict[str(rm_fcount)] = rm_list
            rm_count = 0
            rm_list = []
            rm_fcount +=1

    for k, v in rm_dict.items():
        try:
            df['y_next'].iloc[v] = t_dict[str(int(k)+1)]
        except: 
            df['y_next'].iloc[v] = np.nan
            
    return df
        
    
make_y_next(df, 20, 'ERA')


,VS_T_ID,HEADER_NO,TB_SC,CG_CK,WLS,HOLD,INN2,BF,PA,AB,...,H1,year,month,FIP,BABIP,SLG,OBA,W_OPS,ERA,y_next
G_ID,,,,,,,,,,,,,,,,,,,,,
20160401OBSS0,6,0,0,0,1,0,27,147,37,31,...,6,2016,4,6.755556,0.222222,0.333333,0.451613,0.384194,4.00,5.92136
20160402OBSS0,6,0,0,0,2,0,27,160,42,38,...,5,2016,4,4.644444,0.354839,0.380952,0.552632,0.454774,6.00,5.92136
20160405SSKT0,2,0,1,0,1,0,24,159,42,32,...,6,2016,4,6.325000,0.370370,0.452381,0.562500,0.499732,9.00,5.92136
20160406SSKT0,2,0,1,0,2,0,27,138,38,36,...,2,2016,4,6.866667,0.148148,0.236842,0.500000,0.350000,6.00,5.92136
20160407SSKT0,2,0,1,0,2,1,27,139,34,30,...,3,2016,4,2.533333,0.190476,0.235294,0.166667,0.205784,1.00,5.92136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20161003LGSS0,3,0,0,0,1,0,27,188,46,40,...,9,2016,10,6.088889,0.393939,0.422222,0.625000,0.509417,8.00,2.66667
20161004LGSS0,3,0,0,0,2,1,27,148,35,32,...,7,2016,10,4.977778,0.318182,0.323529,0.468750,0.385974,4.00,NaN
20161005HTSS0,1,0,0,0,1,0,27,140,38,32,...,8,2016,10,2.866667,0.423077,0.400000,0.437500,0.416125,3.00,NaN


### 타자

In [20]:
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

def team_hitter_eda(year, team_name):
    year_index = year - 2016 
    
    # team_hitter data에 해당 경기의 해당 팀의 홈 어웨이 여부 column 추가
    data_games_year = data_games[year_index]
    data_games_year = data_games_year[['G_ID', 'VISIT_KEY', 'HOME_KEY']]
    data_games_year = data_games_year.set_index('G_ID')
    
    data_team_hitter_year = data_team_hitter[year_index]
    data_team_hitter_year = pd.merge(data_team_hitter_year, data_games_year, how='left', on=['G_ID'])

    data_team_hitter_year_team = data_team_hitter_year[data_team_hitter_year.T_ID == team_name]
    df = data_team_hitter_year_team
    df['HOME_KEY'] = df['HOME_KEY'].map(lambda x: 1 if x == team_name else 0 )
    df = df.drop(columns = ['VISIT_KEY', 'G_ID'])
    
    df.reset_index(inplace=True, drop=True) #index 재정렬
    
    # LabelEncoding
    encoder = LabelEncoder()

    df['VS_T_ID'] = encoder.fit_transform(df['VS_T_ID']) #상대편
    df['TB_SC'] = encoder.fit_transform(df['TB_SC']) #이닝 초/말
    
    
    ### 이부분은 모든 EDA에서 동일하게 작성
    
    # GDAY_DS => Datetime type
    df['GDAY_DS'] = df['GDAY_DS'].astype(str) + (df['HEADER_NO']+1).astype(str)
    df['year']=df['GDAY_DS'].str.slice(0,4).astype(int)
    df['month']=df['GDAY_DS'].str.slice(4,6).astype(int)
    df['H1']=df['HIT']-df['H2']-df['H3']-df['HR']
    dt = []
    for i in df['GDAY_DS']:
        dt_ = datetime.strptime(i, '%Y%m%d%H')
        dt.append(dt_)
    df['GDAY_DS'] = pd.Series(dt)
    df['month']=df['GDAY_DS'].apply(lambda x: x.month)
    
    #장타율
    df['SLG']=(df['HIT']+df['HP']+df['BB'])/(df['AB']+df['BB']+df['HP']+df['SF'])
    #출루율
    df['OBA']=(df['H1']+2*df['H2']+3*df['H3']+4*df['HR'])/df['AB']
    df['W_OPS']=0.57* df['SLG']+0.43*df['OBA']
    
    
    
    df = df.drop(columns=['T_ID'])
    #lags
    #lags=[1,4,6,12,30,60]
    #for lag in lags:
    #    df['AVG_lag_'+str(lag)]=df['AVG'].shift(lag).astype(np.float16)
    #SK Expanding window 추가
    #df['expanding_AVG_mean']=df['AVG'].transform(lambda x: x.expanding(2).mean().astype(np.float16))
    #rolling window
    #df['rolling_AVG_mean']=df['AVG'].transform(lambda x: x.rolling(window=7).mean().astype(np.float16))
    #trend
    
    # 경기당 타율 column 생성
    df['AVG'] = df['HIT'] / df['AB']
    
    df['avg_AVG'] = df['AVG'].mean()
    df['AVG_trend'] = (df['AVG'] - df['avg_AVG']).astype(np.float16)
    df.drop(['avg_AVG'],axis=1,inplace=True)
    
    # Drop Categorical feature for 시계열 => coint 과정에서 singular matrix 발생
    df = df.drop(columns=[ 'HOME_KEY'])
    

#     df = df.drop(columns=['T_ID', 'HEADER_NO', 'CG_CK', 'BK'])
#     df = df.drop(columns=['TB_SC', 'HR', 'SB', 'VS_T_ID', 'HOME_KEY', 'HOLD', 'INN2', 'BF', 'CS', 'SH', 'HP', 'GD', 'ERR' ,'ER'])
    df_year_team_name = df
    
    return df_year_team_name

In [ ]:
def team_win_eda(year, team_name):
    year_index = year - 2016 
    
    # team_pitcher data에 해당 경기의 해당 팀의 홈 어웨이 여부 column 추가
    data_games_year = data_games[year_index]
    data_games_year = data_games_year[['G_ID', 'VISIT_KEY', 'HOME_KEY']]
    data_games_year = data_games_year.set_index('G_ID')
    
    data_team_pitcher_year = data_team_pitcher[year_index]
    data_team_pitcher_year = pd.merge(data_team_pitcher_year, data_games_year, how='left', on=['G_ID'])

    data_team_pitcher_year_team = data_team_pitcher_year[data_team_pitcher_year.T_ID == team_name]
    df = data_team_pitcher_year_team
    df['HOME_KEY'] = df['HOME_KEY'].map(lambda x: 1 if x == team_name else 0 )
    df = df.drop(columns = ['VISIT_KEY', 'GDAY_DS'])
    
    # LabelEncoding
    encoder = LabelEncoder()

    df['VS_T_ID'] = encoder.fit_transform(df['VS_T_ID']) #상대편
    df['TB_SC'] = encoder.fit_transform(df['TB_SC']) #이닝 초/말
    #df['WLS'] = encoder.fit_transform(df['WLS']) #승패무 여부
    
    
    ### 이부분은 모든 EDA에서 동일하게 작성
    
    df['H1']=df['HIT']-df['H2']-df['H3']-df['HR']

    #df['G_ID'] = pd.Series(dt)
    df['year']=df['G_ID'].str.slice(0,4).astype(int)
    df['month']=df['G_ID'].str.slice(4,6).astype(int)
    
    df["FIP"] = ((13*df.HR+3*(df.BB+df.HP-df.IB)-2*df.KK)/(df.INN2/3)) +3.2
    df["BABIP"] = (df.HIT-df.HR)/(df.AB-df.KK-df.HR+df.SF)
    df['SLG']=(df['HIT']+df['HP']+df['BB'])/(df['AB']+df['BB']+df['HP']+df['SF'])
    df['OBA']=(df['H1']+2*df['H2']+3*df['H3']+4*df['HR'])/df['AB']
    df['W_OPS']=0.57* df['SLG']+0.43*df['OBA']

    # 시계열 데이터(경기당 방어율 계산)이기 때문에 경기 코드를 index로
    df = df.set_index('G_ID')

    # 경기당 방어율 column 생성
    df['ERA'] = df['ER'] * 27 / df['INN2']

    df = df.drop(columns=['T_ID'])
    df_year_team_name = df
    
    return df_year_team_name
        

### Util

In [21]:
from IPython.display import display
import itertools
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing
import sys

import warnings
warnings.filterwarnings(action='ignore')

In [22]:
from sklearn.metrics import make_scorer

def mae(prediction, correct):
    prediction = np.array(prediction)
    correct = np.array(correct)
    
    difference = correct - prediction
    abs_val = abs(difference)
    
    score = abs_val.mean()
    
    return score

mae_scorer = make_scorer(mae)
mae_scorer

make_scorer(mae)

In [23]:
def mse(prediction, correct):
    prediction = np.array(prediction)
    correct = np.array(correct)
    
    difference = correct - prediction
    sqr_val = difference ** 2
    
    score = sqr_val.mean()
    
    return score

mse_scorer = make_scorer(mse)
mse_scorer

make_scorer(mse)

In [24]:
def plot_train_test_pred_graph(trainset, testset, pred):
    plt.figure(figsize=(15,3))
    plt.plot(trainset.ERA, label='train')
    plt.plot(testset.ERA, label='test')
    plt.plot(testset.index, pred, label='prediction')
    plt.legend()
    plt.show()

## LGBM Class

In [25]:
# 파라미터 지정

params_grid = {
    'num_leaves': [5,10,20,30],
    'reg_alpha': [0.1, 0.5],
    'min_data_in_leaf': [10,30, 50, 100],
    'lambda_l1': [0, 1, 1.5],
    'lambda_l2': [0, 1]
    }

In [26]:
class LGBM_pitcher:
    def __init__(self, eda,  team, yn_range):
        self.eda = eda
        self.team = team
        self.yn_range = yn_range
    
    def run(self):
        df_2016 = self.eda(2016, self.team)
        df_2017 = self.eda(2017, self.team)
        df_2018 = self.eda(2018, self.team)
        df_2019 = self.eda(2019, self.team)
        df_2020 = self.eda(2020, self.team)
        
        df_all = pd.concat([df_2016,df_2017,df_2018,df_2019,df_2020])
        df_all =df_all.reset_index()

        df_all = make_y_next(df_all, self.yn_range, 'ERA')
        
        # 테스트용 데이터 뽑기
        final_test = df_all[df_all['y_next'].isnull()]; 

        df_all.dropna(inplace=True)#일단 드랍하고 진행

        df_all=df_all.drop(['G_ID'],axis=1)

        # 범주형 변수 지정
        cat_features = ['VS_T_ID' ,'HEADER_NO', 'TB_SC' ]
        df_all[cat_features] = df_all[cat_features].astype('category')

        # X,y Train split
        X = df_all.drop(columns = ['y_next'])
        y = df_all['y_next']

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15, shuffle = True, random_state = 2020)

        X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
        X_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_val.columns]


        lgb = LGBMRegressor(boosting_type='gbdt', num_boost_round=2000, learning_rate=0.01)
        lgb_grid = GridSearchCV(estimator=lgb,
                            param_grid=params_grid,
                            n_jobs=10,
                            verbose=3)
        lgb_grid.fit(X_train,y_train) 

        final_test = final_test.drop(['G_ID','y_next'],axis=1)
        final_test[cat_features] = final_test[cat_features].astype('category')

        final_predict = lgb_grid.predict(final_test)

        return final_predict

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
class LGBM_hitter:
    def __init__(self, eda,  team, yn_range):
        self.eda = eda
        self.team = team
        self.yn_range = yn_range
    
    def run(self):
        df_2016 = self.eda(2016, self.team)
        df_2017 = self.eda(2017, self.team)
        df_2018 = self.eda(2018, self.team)
        df_2019 = self.eda(2019, self.team)
        df_2020 = self.eda(2020, self.team)
        
        df_all = pd.concat([df_2016,df_2017,df_2018,df_2019,df_2020])
        df_all =df_all.reset_index()

        df_all = make_y_next(df_all, self.yn_range, 'ERA')
        
        # 테스트용 데이터 뽑기
        final_test = df_all[df_all['y_next'].isnull()]; 

        df_all.dropna(inplace=True)#일단 드랍하고 진행

        df_all=df_all.drop(['G_ID'],axis=1)

        # 범주형 변수 지정
        cat_features = ['VS_T_ID' ,'HEADER_NO', 'TB_SC' ]
        df_all[cat_features] = df_all[cat_features].astype('category')

        # X,y Train split
        X = df_all.drop(columns = ['y_next'])
        y = df_all['y_next']

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15, shuffle = True, random_state = 2020)

        X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
        X_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_val.columns]


        lgb = LGBMRegressor(boosting_type='gbdt', num_boost_round=2000, learning_rate=0.01)
        lgb_grid = GridSearchCV(estimator=lgb,
                            param_grid=params_grid,
                            n_jobs=10,
                            verbose=3)
        lgb_grid.fit(X_train,y_train) 

        final_test = final_test.drop(['G_ID','y_next'],axis=1)
        final_test[cat_features] = final_test[cat_features].astype('category')

        final_predict = lgb_grid.predict(final_test)

        return final_predict

### 승률부분 모델링 방법
- ver.1 : 득점, 실점  각각 예측해서 득점/실점 비교하여 승패 확률 예측 후 승률(이긴경기수/총경기수)계산
- ver.2 : win 1 draw 0.5 lose 0 으로 줘서 모델링 후 승률(이긴경기수/총경기수)계산

ver1. 득점 실점 계산 후 예측

In [40]:
class LGBM_lose:
    def __init__(self, eda,  team, yn_range):
        self.eda = eda
        self.team = team
        self.yn_range = yn_range
        
    def run(self):
        df_2016 = self.eda(2016, self.team)
        df_2017 = self.eda(2017, self.team)
        df_2018 = self.eda(2018, self.team)
        df_2019 = self.eda(2019, self.team)
        df_2020 = self.eda(2020, self.team)
        
        df_all = pd.concat([df_2016,df_2017,df_2018,df_2019,df_2020])
        df_all =df_all.reset_index()

        df_all = make_y_next(df_all, self.yn_range, 'R')
        
        # 테스트용 데이터 뽑기
        final_test = df_all[df_all['y_next'].isnull()]; 

        df_all.dropna(inplace=True)#일단 드랍하고 진행

        df_all=df_all.drop(['G_ID'],axis=1)

        # 범주형 변수 지정
        cat_features = ['VS_T_ID' ,'HEADER_NO', 'TB_SC' ]
        df_all[cat_features] = df_all[cat_features].astype('category')

        # X,y Train split
        X = df_all.drop(columns = ['y_next'])
        y = df_all['y_next']

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15, shuffle = True, random_state = 2020)

        X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
        X_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_val.columns]


        lgb = LGBMRegressor(boosting_type='gbdt', num_boost_round=2000, learning_rate=0.01)
        lgb_grid = GridSearchCV(estimator=lgb,
                            param_grid=params_grid,
                            n_jobs=10,
                            verbose=3)
        lgb_grid.fit(X_train,y_train) 

        final_test = final_test.drop(['G_ID','y_next'],axis=1)
        final_test[cat_features] = final_test[cat_features].astype('category')

        final_predict = lgb_grid.predict(final_test)

        return final_predict

In [41]:
class LGBM_score:
    def __init__(self, eda,  team, yn_range):
        self.eda = eda
        self.team = team
        self.yn_range = yn_range
    def run(self):
        df_2016 = self.eda(2016, self.team)
        df_2017 = self.eda(2017, self.team)
        df_2018 = self.eda(2018, self.team)
        df_2019 = self.eda(2019, self.team)
        df_2020 = self.eda(2020, self.team)

        df_all = pd.concat([df_2016,df_2017,df_2018,df_2019,df_2020])
        df_all =df_all.reset_index()
        
        df_all = make_y_next(df_all, self.yn_range, 'RUN')
        
        # 테스트용 데이터 뽑기
        final_test = df_all[df_all['y_next'].isnull()]; 

        df_all.dropna(inplace=True)#일단 드랍하고 진행

        df_all = df_all.drop(['GDAY_DS'],axis=1)

        # 범주형 변수 지정
        cat_features = ['VS_T_ID' ,'HEADER_NO', 'TB_SC' ]
        df_all[cat_features] = df_all[cat_features].astype('category')

        # X,y Train test val split
        X = df_all.drop(columns = ['y_next'])
        y = df_all['y_next']

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15, shuffle = True, random_state = 2020)

        X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
        X_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_val.columns]


        lgb = LGBMRegressor(boosting_type='gbdt', num_boost_round=2000, learning_rate=0.01)
        lgb_grid = GridSearchCV(estimator=lgb,
                            param_grid=params_grid,
                            n_jobs=10,
                            verbose=3)
        lgb_grid.fit(X_train,y_train) 

        final_test = final_test.drop(['GDAY_DS','y_next'],axis=1)
        final_test[cat_features] = final_test[cat_features].astype('category')

        final_predict = lgb_grid.predict(final_test)

        return final_predict


ver2. lgbm으로 승패무 예측

In [ ]:
class LGBM_win:
    def __init__(self, eda,  team, yn_range):
        self.eda = eda
        self.team = team
        self.yn_range = yn_range
    
    def run(self):
        df_2016 = self.eda(2016, self.team)
        df_2017 = self.eda(2017, self.team)
        df_2018 = self.eda(2018, self.team)
        df_2019 = self.eda(2019, self.team)
        df_2020 = self.eda(2020, self.team)
        
        df_all = pd.concat([df_2016,df_2017,df_2018,df_2019,df_2020])
        df_all =df_all.reset_index()

        df_all = make_y_next(df_all, self.yn_range, 'WLS')
        
        # 테스트용 데이터 뽑기
        final_test = df_all[df_all['y_next'].isnull()]; 

        df_all.dropna(inplace=True)#일단 드랍하고 진행

        df_all=df_all.drop(['G_ID'],axis=1)

        # 범주형 변수 지정
        cat_features = ['VS_T_ID' ,'HEADER_NO', 'TB_SC' ]
        df_all[cat_features] = df_all[cat_features].astype('category')

        # X,y Train split
        X = df_all.drop(columns = ['y_next'])
        y = df_all['y_next']

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15, shuffle = True, random_state = 2020)

        X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
        X_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_val.columns]


        lgb = LGBMRegressor(boosting_type='gbdt', num_boost_round=2000, learning_rate=0.01)
        lgb_grid = GridSearchCV(estimator=lgb,
                            param_grid=params_grid,
                            n_jobs=10,
                            verbose=3)
        lgb_grid.fit(X_train,y_train) 

        final_test = final_test.drop(['G_ID','y_next'],axis=1)
        final_test[cat_features] = final_test[cat_features].astype('category')

        final_predict = lgb_grid.predict(final_test)

        return final_predict

# 최종 실행

In [30]:
team_list = ['WO','OB','NC','SK','LG','KT','LT','HT','SS','HH']

### 타율

In [32]:
for t in team_list:
    obj = LGBM_hitter(team_hitter_eda, t, 20)
    predict = obj.run()
    print(predict)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   19.2s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:  1.1min
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:  2.0min
[Parallel(n_jobs=10)]: Done 492 tasks      | elapsed:  2.5min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  2.9min
[Parallel(n_jobs=10)]: Done 960 out of 960 | elapsed:  3.1min finished


[0.22336694 0.21964685]
Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    5.6s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:   53.5s
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:  1.7min
[Parallel(n_jobs=10)]: Done 492 tasks      | elapsed:  2.2min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  2.6min
[Parallel(n_jobs=10)]: Done 960 out of 960 | elapsed:  2.9min finished


[0.30757816 0.30928145 0.30581315 0.30700643 0.30632876 0.30587121
 0.30706851 0.30750622 0.3078314  0.30867325 0.30675131 0.30978595
 0.3085539  0.30832511 0.30550988 0.30464305 0.3054757  0.30214179
 0.3035796  0.30296866]
Fitting 5 folds for each of 192 candidates, totalling 960 fits


...

[0.27480083 0.27240112 0.27182166 0.27269275 0.25515693 0.25577182
 0.25586354 0.2560635  0.2556077  0.2547003  0.25437109 0.255155
 0.25606041 0.25487429 0.25548897 0.25431779 0.25472388 0.25514097]
Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    5.4s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:   43.3s
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:  1.9min
[Parallel(n_jobs=10)]: Done 492 tasks      | elapsed:  2.3min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  2.8min
[Parallel(n_jobs=10)]: Done 960 out of 960 | elapsed:  3.2min finished


[0.3029571  0.30242409 0.30281084 0.30216337 0.30096578 0.30107845
 0.3012557  0.30033205 0.30058393 0.30075413 0.30057392 0.30038466
 0.30104802 0.30071935 0.30116958 0.30029755 0.30142961 0.30018232
 0.29967764 0.30144874 0.30106905]
Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    8.1s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:  1.1min
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:  1.9min
[Parallel(n_jobs=10)]: Done 492 tasks      | elapsed:  2.4min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  3.1min
[Parallel(n_jobs=10)]: Done 960 out of 960 | elapsed:  3.4min finished


[0.24733862 0.24796208 0.24783797 0.24745267 0.25327967 0.25259805
 0.25320315 0.25330076 0.25269686 0.25287751 0.25333413 0.25302615
 0.25370478 0.25339759 0.25341316 0.25319941 0.25338324 0.2530364
 0.25279088 0.25349538 0.25457744]


### 방어율

In [33]:
for t in team_list:
    print(t)
    obj = LGBM_pitcher(team_pitcher_eda, t, 20)
    predict = obj.run()
    print(predict)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    5.4s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:   46.4s
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:  2.1min
[Parallel(n_jobs=10)]: Done 492 tasks      | elapsed:  3.3min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  4.9min
[Parallel(n_jobs=10)]: Done 960 out of 960 | elapsed:  6.0min finished


[2.09526489 1.69505411]
Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    5.5s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:   52.0s
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:  1.9min
[Parallel(n_jobs=10)]: Done 492 tasks      | elapsed:  2.9min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  4.3min
[Parallel(n_jobs=10)]: Done 960 out of 960 | elapsed:  5.3min finished


[4.56066696 4.75994469 4.77290257 4.71477348 4.77687009 4.62940151
 4.76238394 4.63339848 4.70537203 4.48498487 4.35859241 4.91557136
 4.95992136 4.94289209 4.5859663  4.30819727 4.44475068 4.81826301
 4.72271455 4.9883872 ]
Fitting 5 folds for each of 192 candidates, totalling 960 fits


...

[5.1521789  4.99757728 4.59357169 4.21026085 4.72626459 4.19042111
 4.73892873 4.01360293 4.68512537 4.18689369 4.68189269 4.87893147
 4.44915144 4.26470768 4.33965884 4.43587342 5.00944694 4.27454822]
Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    5.6s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:   57.8s
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:  2.4min
[Parallel(n_jobs=10)]: Done 492 tasks      | elapsed:  4.0min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  5.9min
[Parallel(n_jobs=10)]: Done 960 out of 960 | elapsed:  6.7min finished


[4.86885972 4.7746557  4.83281023 5.14295901 4.82327664 4.87265102
 5.10088837 4.83594347 5.10337733 4.80856983 4.92991388 4.91476873
 4.95510371 4.86492944 4.73310008 4.81284563 4.84674776 5.0520479
 4.84759655 4.94384795 4.94970796]
Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    6.5s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:  1.1min
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:  2.1min
[Parallel(n_jobs=10)]: Done 492 tasks      | elapsed:  4.7min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  8.8min
[Parallel(n_jobs=10)]: Done 960 out of 960 | elapsed: 11.0min finished


[4.8787794  4.60648501 4.83464351 4.79791869 4.45211592 4.57261381
 4.69744293 4.85074704 4.72323236 4.44829723 4.65884273 4.84000681
 4.81393657 4.60616391 4.80815604 4.6340241  4.82075723 4.69562395
 4.81638754 4.65329571 4.66129892]


### 승률 ver1.

실행 개오래걸림

In [43]:
for t in team_list:
    print(t)
    
    score = LGBM_score(team_hitter_eda, t, 20)
    lose = LGBM_lose(team_pitcher_eda, t, 20)
    
    lose = lose.run()
    score = score.run()
    
    score = np.array(score)
    lose = np.array(lose)
    win_rate= np.square(score)  / ( np.square(score)  +   np.square(lose) )
    print(win_rate)

WO
Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   24.7s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:  1.3min
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:  2.4min
[Parallel(n_jobs=10)]: Done 492 tasks      | elapsed:  3.6min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  5.7min
[Parallel(n_jobs=10)]: Done 960 out of 960 | elapsed:  6.7min finished


Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    9.8s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:  1.1min
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:  2.1min
[Parallel(n_jobs=10)]: Done 492 tasks      | elapsed:  3.5min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  5.5min
[Parallel(n_jobs=10)]: Done 960 out of 960 | elapsed:  6.5min finished


[0.79158127 0.8914123 ]
OB
Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   10.2s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:   56.4s
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:  2.0min
[Parallel(n_jobs=10)]: Done 492 tasks      | elapsed:  3.3min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  6.4min
[Parallel(n_jobs=10)]: Done 960 out of 960 | elapsed:  8.7min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.


Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   27.7s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:  1.7min
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:  3.5min
[Parallel(n_jobs=10)]: Done 492 tasks      | elapsed:  5.3min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  8.4min
[Parallel(n_jobs=10)]: Done 960 out of 960 | elapsed: 10.2min finished


KeyboardInterrupt: 

### 승률 ver2.

In [ ]:
for t in team_list:
    print(t)
    obj = LGBM_win(team_pitcher_eda, t, 20)
    predict = obj.run()
    print(predict)

In [ ]:
class LGBM_Experiment: #팀별로 잔여경기가 다르니 그냥 안하는게 나을수도?
    def __init__(self, eda, model):
        super(Experiment, self).__init__()
        
        self.eda = eda
        self.model = model
    
    def run(self):
        model_key = 'Pred.{}.{}'.format(self.target, self.model.__name__)
        test_key = 'Test.{}'.format(self.target)
        dict = {'team': [], test_key: [], model_key : []}

        #2016~2019, OB, LG, WO, SK, KT, HH, HT, SS, LT, NC

        for team in ['OB', 'LG', 'WO', 'SK', 'KT', 'HH', 'HT', 'SS', 'LT', 'NC']:

            try:
                model_obj = self.model(self.eda, team)
                model_test, model_pred = model_obj.run()
                dict['team'].append(team)

                dict[test_key].append(model_test)
                dict[model_key].append(model_pred)
            except:
                print(year, team)
                continue
                    
        
        df = pd.DataFrame(dict)
        
        #모델 오차의 절대값  
        difference_column = '{}_abs_difference'.format(self.model.__name__)
        df[difference_column] = abs(df[test_key]-df[model_key])
        model_difference_mean =  df[difference_column].mean()
        model_difference_std = df[difference_column].std()
        
        print("{} 모델의 오차 절대값 평균:".format(self.model.__name__), model_difference_mean)
        print("{} 모델의 오차 절대값 표준편차:".format(self.model.__name__), model_difference_std)
        display(df)
        
        return model_difference_mean, model_difference_std, df

In [ ]:
pitcher_experiment = LGBM_Experiment(team_pitcher_eda, LGBM_pitcher)
hitter_experiment = LGBM_Experiment(team_hitter_eda, LGBM_hitter)